In [65]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from dateutil import parser
import pymap3d as pm3d
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from scipy.spatial.transform import Rotation
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


In [66]:
input_file = "./100m_standoff_1346.jsonl"
origin = {'lat': 30.215906, 'lon': -97.760757}
START_TIME = pd.Timestamp('2023-11-07T19:49:00.000000+0000')
END_TIME = pd.Timestamp('2023-11-07T19:51:00.000000+0000')

In [67]:
# empty dataframe with 2 columns 
gps_df = pd.DataFrame(columns=['ts', 'boat_lat', 'boat_lon'])
ais_df = pd.DataFrame(columns=['ts', 'buoy_lat', 'buoy_lon'])
imu_df = pd.DataFrame(columns=['ts', 'boat_yaw', 'boat_pitch', 'boat_roll'])
cv_df = pd.DataFrame(columns=['ts', 'cv_range', 'cv_az', 'class_id'])

In [68]:
# parse json lines
with open(input_file) as f:
    lines = f.readlines()
    for line in lines:
        try:
            rowdata = json.loads(line)
            ts = parser.parse(rowdata['ts'])
            if ts < START_TIME:
                continue
            if ts > END_TIME:
                continue
        except:
            continue
        if "CVFrame" in line:
            try:
                if rowdata['msg']['CVFrame']['detections'] == None:
                    continue
                for det in rowdata['msg']['CVFrame']['detections']:
                    row = {'ts': parser.parse(rowdata['msg']['CVFrame']['capture_timestamp']), 'cv_range': det['depth'], 'cv_az': det['rel_angle'], 'class_id': det['class_id']}
                    cv_df = cv_df.append(row, ignore_index=True)
            except:
                continue
        elif "Imu" in line:
            try:
                row = {'ts': ts, 'boat_yaw': rowdata['msg']['Imu']['yaw'], 'boat_pitch': rowdata['msg']['Imu']['pitch'], 'boat_roll': rowdata['msg']['Imu']['roll']}
                imu_df = imu_df.append(row, ignore_index=True)
            except:
                continue
        elif "FilteredGps" in line:
            try:
                row = {'ts': ts, 'boat_lat': rowdata['msg']['FilteredGps']['location']['lat'], 'boat_lon': rowdata['msg']['FilteredGps']['location']['lon']}
                gps_df = gps_df.append(row, ignore_index=True)
            except:
                continue
        elif "Ais" in line:
            try:
                row = {'ts': ts, 'buoy_lat': rowdata['msg']['Ais']['lat'], 'buoy_lon': rowdata['msg']['Ais']['lon']}
                ais_df = ais_df.append(row, ignore_index=True)
            except:
                continue

ais_df.dropna(inplace=True)
ais_df.set_index('ts', inplace=True)

# Resample to a higher frequency (e.g., every minute) and interpolate missing values
ais_resampled = ais_df.resample('50ms').last().interpolate(method='time', limit=30, limit_direction='forward', axis=0)
    
# Reset the index if you want a regular column for timestamps
ais_resampled.reset_index(inplace=True)


In [69]:
print(len(cv_df))
print(len(imu_df))
print(len(gps_df))
print(len(ais_resampled))

355
2125
2132
2397


In [70]:
# remove cv_df rows with duplicate cv_az values
cv_df_no_coast = cv_df.drop_duplicates(subset=['cv_az'])
print(len(cv_df_no_coast))
print(len(cv_df))

203
355


In [71]:
#print(len(ais_resampled))
#merged_depth = pd.merge_asof(cv_df_no_coast.sort_values(by='ts'), ais_resampled.sort_values(by='ts'), on='ts', direction='nearest', tolerance=pd.Timedelta(milliseconds=100))
#merged_depth.dropna(inplace=True)
#print(len(merged_depth))

print(len(merged_depth))
merged_gps = pd.merge_asof(merged_depth.sort_values(by='ts'), gps_df.sort_values(by='ts'), on='ts', direction='nearest', tolerance=pd.Timedelta(milliseconds=100))
merged_gps.dropna(inplace=True)
print(len(merged_gps))

merged = pd.merge_asof(merged_gps, imu_df.sort_values(by='ts'), on='ts', direction='nearest', tolerance=pd.Timedelta(milliseconds=100))
merged.dropna(inplace=True)
print(len(merged))


gps_buoy_merged = pd.merge_asof(gps_df.sort_values(by='ts'), ais_resampled.sort_values(by='ts'), on='ts', direction='nearest', tolerance=pd.Timedelta(milliseconds=100))
gps_buoy_merged.dropna(inplace=True)
print(len(gps_buoy_merged))

gps_buoy_imu_merged = pd.merge_asof(imu_df.sort_values(by='ts'), gps_buoy_merged.sort_values(by='ts'), on='ts', direction='nearest', tolerance=pd.Timedelta(milliseconds=100))
gps_buoy_imu_merged.dropna(inplace=True)
print(len(gps_buoy_imu_merged))

209
0
0
2132
2125


In [72]:
merged.head()

,cv_range,cv_az,class_id,buoy_lat,buoy_lon,boat_lat,boat_lon,ts,boat_yaw,boat_pitch,boat_roll


In [73]:
merged['buoy_a'],merged['buoy_e'],merged['buoy_r'] = pm3d.geodetic2aer(merged['buoy_lat'], merged['buoy_lon'], 0, merged['boat_lat'], merged['boat_lon'], 0)
gps_buoy_imu_merged['buoy_a'],gps_buoy_imu_merged['buoy_e'],gps_buoy_imu_merged['buoy_r'] = pm3d.geodetic2aer(gps_buoy_imu_merged['buoy_lat'], gps_buoy_imu_merged['buoy_lon'], 0, gps_buoy_imu_merged['boat_lat'], gps_buoy_imu_merged['boat_lon'], 0)

gps_buoy_imu_merged['buoy_az'] = ((gps_buoy_imu_merged['buoy_a'] + 180 ) % 360) - ((np.rad2deg(gps_buoy_imu_merged['boat_yaw']) + 180) %360)
merged['buoy_az'] = ((merged['buoy_a'] + 180 ) % 360) - ((np.rad2deg(merged['boat_yaw']) + 180) %360)





In [74]:
with open("buoy_data_100m.jsonl", "w") as f:
    f.write(gps_buoy_imu_merged.to_json(date_format='iso'))

In [33]:
# plot e and n on the same plot
fig333 = go.Figure()
fig333.add_trace(go.Scatter(x=gps_buoy_imu_merged['ts'], y=(gps_buoy_imu_merged['buoy_a'] + 180 ) % 360, 
                    mode='markers',
                    name='buoy az'))
fig333.add_trace(go.Scatter(x=merged['ts'], y=(np.rad2deg(merged['boat_yaw']) + 180) %360 , 
                    mode='markers', hovertext=merged['class_id'],
                    name='boat yaw'))

fig333.update_xaxes(title_text='Time (UTC)')
fig333.update_yaxes(title_text='az (deg)')
fig333.update_layout(title_text='Depth error')
fig333.show()

fig334 = go.Figure()
fig334.add_trace(go.Scatter(x=gps_buoy_imu_merged['ts'], y=gps_buoy_imu_merged['buoy_az'], 
                    mode='markers', hovertext=gps_buoy_imu_merged['buoy_r'],
                    name='buoy az'))
fig334.add_trace(go.Scatter(x=cv_df_no_coast['ts'], y= cv_df_no_coast['cv_az'] - 36, 
                    mode='markers', hovertext=merged['cv_range'],
                    name='cv az'))

fig334.update_xaxes(title_text='Time (UTC)')
fig334.update_yaxes(title_text='az (deg)')
fig334.update_layout(title_text='Azimuth angle in boat body frame')
fig334.show()
# fig333.add_trace(go.Scatter(x=merged['ts'], y=(merged['boat_yaw']+ 180) % 360 + merged['cv_az'] - 36 , 
#                     mode='markers', hovertext=merged['class_id'],
#                     name='cv_az'))








In [34]:
# plot e and n on the same plot
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=gps_buoy_merged['ts'], y=gps_buoy_imu_merged['buoy_r'], 
                    mode='markers',
                    name='true depth'))
fig3.add_trace(go.Scatter(x=merged['ts'], y=merged['cv_range'], 
                    mode='markers',
                    name='zed measured depth', hovertext=merged['class_id']))



fig3.update_xaxes(title_text='Time (UTC)')
fig3.update_yaxes(title_text='depth (m)')
fig3.update_layout(title_text='Depth error')
fig3.show()




In [35]:
merged['az_err'] = merged['buoy_az'] - merged['cv_az']
merged['depth_err'] = merged['buoy_r'] - merged['cv_range']


In [36]:
# plot e and n on the same plot
fig31 = go.Figure()
fig31.add_trace(go.Scatter(x=merged['ts'], y=merged['depth_err'], 
                    mode='markers',
                    name='r_error'))

fig31.update_xaxes(title_text='Time (UTC)')
fig31.update_yaxes(title_text='depth error (m)')
fig31.update_layout(title_text='Depth error')
fig31.show()

# plot e and n on the same plot
fig30 = go.Figure()
fig30.add_trace(go.Scatter(x=merged['ts'], y=merged['az_err']+36, 
                    mode='markers',
                    name='az error'))

fig30.update_xaxes(title_text='Time (UTC)')
fig30.update_yaxes(title_text='az error (deg)')
fig30.update_layout(title_text='az error')
fig30.show()



In [37]:
# plot e and n on the same plot
fig31 = go.Figure()
fig31.add_trace(go.Scatter(x=merged['buoy_r'], y=merged['depth_err'], 
                    mode='markers',
                    name='r_error'))

fig31.update_xaxes(title_text='Buoy Range (m)')
fig31.update_yaxes(title_text='depth error (m)')
fig31.update_layout(title_text='Depth error')
fig31.show()

# plot e and n on the same plot
fig30 = go.Figure()
fig30.add_trace(go.Scatter(x=merged['buoy_r'], y=merged['az_err']+36, 
                    mode='markers',
                    name='az error'))

fig30.update_xaxes(title_text='Buoy Range (m)')
fig30.update_yaxes(title_text='az error (deg)')
fig30.update_layout(title_text='az error')
fig30.show()